# Udacity_project_01

## Workspace setup

In [ ]:
from azureml.core import Workspace, Experiment

# Get the existing workspace
ws = Workspace.from_config()#get(name="quick-starts-ws-139918", resource_group="aml-quickstarts-139918" )

# Verify workspace attributes
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

## Experiment setup

In [ ]:
# Create the experiment
exp = Experiment(workspace=ws, name="Udacity_project_01_Exp01")
run = exp.start_logging()

## ComputeTarget setup

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.exceptions import ComputeTargetException

# Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

# Name the compute cluster
cluster_name = "MyComputeInstanceJDY"

# Verify that name do not exist already
try:
    comp_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    print('Create new cluster...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    comp_target = ComputeTarget.create(ws, cluster_name, compute_config)

comp_target.wait_for_completion(show_output=True)
print('Cluster Details: ', comp_target.get_status().serialize())

In [ ]:
# Setup environment config with libraries versions
%%writefile conda_dependencies.yml

dependencies:
    - python=3.6.2
    - scikit-learn
    - numpy
    - pip:
        - azureml-defaults

In [ ]:
from azureml.core import Environment
# Create environment
sklearn_env= Environment.from_conda_specification(name='sklearn_env', file_path='conda_dependencies.yml')

## Run HyperDrive Config

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.core import ScriptRunConfig
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import RunConfiguration
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
ps = RandomParameterSampling({"C": uniform(0.01, 1), "max_iter": choice(range(10, 200, 25))})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor = 0.2, delay_evaluation=0)

#if "training" not in os.listdir():
#    os.mkdir("./training")

# Create a ScriptRunConfig for use with train.py
est = ScriptRunConfig(source_directory = "./", script="train.py", compute_target=comp_target, environment=sklearn_env)

# Create a HyperDriveConfig using the run_config, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=est,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=12,
                                     max_concurrent_runs=4
                                    )

In [ ]:
os.listdir()

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyper_run = exp.submit(hyperdrive_config)
RunDetails(hyper_run).show()

## Record best model

In [ ]:
import joblib
# Get your best run and save the model from that run.
hd_best_run = hyper_run.get_best_run_by_primary_metric()
hd_best_run

In [ ]:
# Save best model
model=hd_best_run.register_model(model_name='hd_best_run', model_path='./outputs/Best_model.joblib')
print(model.name, model.id, model.version)

## Load data set in DatasetFactory

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory
# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
path='https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
ds = TabularDatasetFactory.from_delimited_files(path=path)

In [ ]:
from train import clean_data
import pandas as pd
# Use the clean_data function to clean your data.
x, y = clean_data(ds)
# Concat X & y-> no train test split is done because the model will use cross validation
data_cleaned = pd.concat([x,y], axis=1)

## Run AutoML Config

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=data_cleaned,
    label_column_name='y',
    n_cross_validations=5)

In [ ]:
# Submit your automl run
automlrun = exp.submit(automl_config, show_output=True)

In [ ]:
# Retrieve and save your best automl model.
atml_best_run = automlrun.get_best_child()
atml_best_run.get_details()

In [ ]:
# Close properly
comp_target.delete()